In [1]:
import pandas as pd
from tqdm import tqdm
import re, os
from os.path import join

account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
memo_dtype = {'채무자키' : str, '메모키':str}
event_dtype = {'채무자키':str, '법조치키':str, '계좌키':str, '관련법조치키':str, '법취하키':str, '타법조치키':str, '타채무자키':str, '관할법원코드':str}
deposit_dtype = {'채무자키' : str, '입금키':str, '계좌키':str, '입금계좌번호':str, '계좌번호':str, '타입금키':str, '타채무자키':str, '보증인키':str}

# 자산확정일 : 채무자, 계좌, 보증인, 개인회생, 신용회복
# 매각일 : 계좌, 입금(계좌조회,계좌별표시,기간제한x), 메모, 법조치(새채무자키)

In [2]:
# 파일읽기 # 매각일 자료도 일단은 dataset에 저장하고 나중에 옮기든지 하자.
################################
company = "솔림"
cutDate = "230731"
sellDate = "230831"
sell_ext = ".xlsx"
################################
cut_ext = ".pkl" # 자산정리하면서 pkl 만들었으니
path_base = join(r"D:\3.자산\전산 dataset", company)

cutFiles = [name + cut_ext for name in ["채무자조회새창_20230731_2106", "계좌조회새창_20230731_2109", "보증인새창_20230731_2110", \
    "개인회생새창_20230731_2115_진행중_계좌키분리_기준데이터", "20230731_신용회복전체리스트_솔림"]]
sellFiles = ["계좌조회새창_20230831_2250", "메모조회새창_20230905_0933", "법조치조회새창_20230831_2254", "입금조회새창_20230905_0939"]

# 자산확정일
cut_dir = join(path_base, cutDate)
debt = pd.read_pickle(join(cut_dir, cutFiles[0]))
cut_accout = pd.read_pickle(join(cut_dir, cutFiles[1]))
grt = pd.read_pickle(join(cut_dir, cutFiles[2]))
rehabilitation = pd.read_pickle(join(cut_dir, cutFiles[3]))
credit = pd.read_pickle(join(cut_dir, cutFiles[4]))

# 매각일
sell_dir = join(path_base, sellDate)
sell_account = pd.read_excel(join(sell_dir, sellFiles[0]+sell_ext)).fillna("")
memo = pd.read_excel(join(sell_dir, sellFiles[1]+sell_ext)).fillna("")
event = pd.read_excel(join(sell_dir, sellFiles[2]+sell_ext)).fillna("")
deposit = pd.read_excel(join(sell_dir, sellFiles[3]+sell_ext)).fillna("")

# 매각리스트
keys = pd.read_excel(r"C:\Users\SL\Desktop\매각건정리.xlsx", sheet_name="P1애플(신복)230831", dtype=str).fillna("")

c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
# 매각일 자료 pkl 저장
sell_account.to_pickle(join(sell_dir, sellFiles[0]+cut_ext))
memo.to_pickle(join(sell_dir, sellFiles[1]+cut_ext))
event.to_pickle(join(sell_dir, sellFiles[2]+cut_ext))
deposit.to_pickle(join(sell_dir, sellFiles[3]+cut_ext))

In [ ]:
# 추출하기 : 법조치는 새채무자키로 추출